In [39]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [41]:
df = pd.read_csv("SP100_vol_ffill.csv", index_col="Date",parse_dates=True)

In [42]:
df.head()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,NKE,NOW,NVDA,ORCL,PEP,PFE,PG,PLTR,PM,PYPL
Date,,,,,,,,,,,,,,,,,,,,,
03-01-2007 00:00,1.974049,0.034057,0.667621,0.822765,0.942692,0.267089,1.618511,1.045328,0.560132,0.825839,...,0.313073,0.109702,1.897743,0.652943,0.343138,0.300183,0.338006,0.06684,0.067433,0.038919
04-01-2007 00:00,1.974049,0.034057,0.667621,0.822765,0.942692,0.267089,1.618511,1.045328,0.560132,0.825839,...,0.313073,0.109702,1.897743,0.652943,0.343138,0.300183,0.338006,0.06684,0.067433,0.038919
05-01-2007 00:00,1.974049,0.034057,0.667621,0.822765,0.942692,0.267089,1.618511,1.045328,0.560132,0.825839,...,0.313073,0.109702,1.897743,0.652943,0.343138,0.300183,0.338006,0.06684,0.067433,0.038919
08-01-2007 00:00,1.974049,0.034057,0.667621,0.822765,0.942692,0.267089,1.618511,1.045328,0.560132,0.825839,...,0.313073,0.109702,1.897743,0.652943,0.343138,0.300183,0.338006,0.06684,0.067433,0.038919
09-01-2007 00:00,1.974049,0.034057,0.667621,0.822765,0.942692,0.267089,1.618511,1.045328,0.560132,0.825839,...,0.313073,0.109702,1.897743,0.652943,0.343138,0.300183,0.338006,0.06684,0.067433,0.038919


In [43]:
train=df.iloc[:int(len(df)*0.7)]

In [44]:
test=df.iloc[int(len(df)*0.7)-22:]

In [45]:
df.iloc[int(len(df)*0.7)]

AAPL    1.063223
ABBV    0.616832
ABT     0.499624
ACN     0.728602
ADBE    0.981625
          ...   
PFE     0.658292
PG      0.426430
PLTR    0.066840
PM      0.548380
PYPL    1.050004
Name: 03-07-2017 00:00, Length: 80, dtype: float64

In [46]:
train=np.array(train)

In [47]:
train.shape

(2643, 80)

In [48]:
RV_d = []
for i in range(len(train)-1):
  RV_d.append(train[i].astype(float))

RV_w = []
for i in range(len(train)-5):
  RV_w.append(np.mean(train[i:i+5]).astype(float))

RV_m = []
for i in range(len(train)-22):
  RV_m.append(np.mean(train[i:i+22]).astype(float))

RV_d=RV_d[len(RV_d)-len(RV_m):]
RV_w=RV_w[len(RV_w)-len(RV_m):]

In [49]:
RV_D = np.zeros([len(RV_d),train.shape[1]])
for i in range(len(RV_d)):
  RV_D[i,:]=RV_d[i]

RV_W = np.zeros([len(RV_w),train.shape[1]])
for i in range(len(RV_w)):
  RV_W[i,:]=RV_w[i]

RV_M = np.zeros([len(RV_m),train.shape[1]])
for i in range(len(RV_m)):
  RV_M[i,:]=RV_m[i]

In [50]:
RV_actuals=train[22:,:]

In [51]:
RV_actuals.shape

(2621, 80)

In [52]:
test=np.array(test)

In [53]:
test.shape

(1156, 80)

In [54]:
for i in range(len(test)-1):
      RV_d.append(test[i].astype(float))

RV_w = []
for i in range(len(test)-5):
  RV_w.append(np.mean(test[i:i+5]).astype(float))

RV_m = []
for i in range(len(test)-22):
  RV_m.append(np.mean(test[i:i+22]).astype(float))

RV_d=RV_d[len(RV_d)-len(RV_m):]
RV_w=RV_w[len(RV_w)-len(RV_m):]
RV_D_test = np.zeros([len(RV_d),test.shape[1]])
for i in range(len(RV_d)):
  RV_D_test[i,:]=RV_d[i]

RV_W_test = np.zeros([len(RV_w),test.shape[1]])
for i in range(len(RV_w)):
  RV_W_test[i,:]=RV_w[i]

RV_M_test = np.zeros([len(RV_m),test.shape[1]])
for i in range(len(RV_m)):
  RV_M_test[i,:]=RV_m[i]

In [57]:
forecasts=np.zeros([test.shape[0]-22,test.shape[1]])
for j in range(test.shape[1]):
  Data ={'Actual': RV_actuals[:,j],
  'D': RV_D[:,j],
  'W': RV_W[:,j],
  'M': RV_M[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  rv_scaled = sm.add_constant(rv_scaled)
  rv_scaled['Actual']=rv_scaled['Actual'].astype(float)
  rv_scaled['D']=rv_scaled['D'].astype(float)
  rv_scaled['W']=rv_scaled['W'].astype(float)
  rv_scaled['M']=rv_scaled['M'].astype(float)
  X = rv_scaled.drop("Actual", axis = 1)
  y = rv_scaled["Actual"]
  results = sm.OLS(y, X).fit()
  Data ={'D': RV_D_test[:,j],
  'W': RV_W_test[:,j],
  'M': RV_M_test[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  # Use the same columns as in training
  X_test = sm.add_constant(rv_scaled[results.model.exog_names[1:]])  # skip 'const' in exog_names

  # Now predict
  forecasts[:,j] = results.predict(X_test)

In [58]:
forecasts.shape

(1134, 80)

In [59]:
forecasts[0,:]

array([1.06635676, 0.63962897, 0.52927652, 0.73953642, 0.92506877,
       0.46708546, 2.95212706, 0.89894751, 0.62151912, 0.96073752,
       1.83824801, 0.51270022, 0.64868546, 1.03722597, 0.6581203 ,
       0.85718545, 0.57205688, 1.08830528, 0.49210327, 0.80808551,
       0.88764034, 1.02645479, 0.47660007, 0.90941451, 0.73221923,
       1.12782835, 1.31487366, 0.93290293, 0.630562  , 1.04175601,
       0.75320541, 0.64184965, 0.50081908, 0.6413659 , 0.44427241,
       1.00974422, 0.81418681, 0.60701596, 0.93239609, 1.06961401,
       0.51291116, 0.99149718, 0.97716623, 0.8382237 , 0.71156017,
       0.44319367, 0.43776063, 0.7314589 , 0.65954804, 0.77840098,
       0.42817591, 0.77514081, 0.41133022, 1.        , 0.55934125,
       0.41120184, 0.8837098 , 0.76787482, 0.57531272, 0.68115526,
       0.50309645, 0.91989966, 0.91840112, 0.45628106, 0.5416173 ,
       0.65595098, 0.88409564, 0.85686689, 0.50619531, 1.41501128,
       1.96492176, 1.29111363, 2.0669553 , 1.44043794, 0.42951

In [60]:
df1=df.iloc[int(len(df)*0.7):]

In [61]:
df1.head()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,NKE,NOW,NVDA,ORCL,PEP,PFE,PG,PLTR,PM,PYPL
Date,,,,,,,,,,,,,,,,,,,,,
03-07-2017 00:00,1.063223,0.616832,0.499624,0.728602,0.981625,0.485834,2.980026,0.891043,0.621440,0.970485,...,2.008477,1.310070,2.161606,1.470019,0.422495,0.658292,0.426430,0.06684,0.548380,1.050004
05-07-2017 00:00,1.038708,0.603610,0.489255,0.726756,1.005544,0.495195,3.274906,0.851366,0.624378,0.999759,...,2.019470,1.325671,2.201794,1.469669,0.419143,0.658447,0.418427,0.06684,0.580678,1.173467
06-07-2017 00:00,1.037928,0.622561,0.595787,0.733791,1.006799,0.490702,3.242977,0.876565,0.620852,1.002347,...,2.022491,1.328324,2.148457,1.480000,0.419074,0.681030,0.421042,0.06684,0.580954,1.216530
07-07-2017 00:00,1.049405,0.620301,0.611813,0.752526,1.018479,0.478165,3.071428,0.874210,0.648642,1.018699,...,2.029117,1.353083,2.177609,1.485053,0.422309,0.669937,0.423958,0.06684,0.584178,1.219856
10-07-2017 00:00,1.049382,0.586922,0.611845,0.752285,1.024031,0.481919,3.083223,0.850630,0.687296,1.052489,...,2.026721,1.381717,2.295564,1.485827,0.446412,0.669554,0.435961,0.06684,0.583519,1.245615


In [62]:
for i in range(len(df1.columns)):
  df1[df1.columns[i]]=forecasts[:,i]

C:\Users\VATSAL\AppData\Local\Temp\ipykernel_15136\337315467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[df1.columns[i]]=forecasts[:,i]


In [63]:
df1.to_csv("HAR Forecasts.csv")

In [64]:
test=df.iloc[int(len(df)*0.7):]
test=np.array(test)
RMSE=np.sqrt(np.sum((test-forecasts)**2)/(test.shape[0]*test.shape[1]))
MAE=(np.sum(abs(test-forecasts))/(test.shape[0]*test.shape[1]))
QLIKE=np.sum(test/forecasts-np.log(test/forecasts)-1)/(test.shape[0]*test.shape[1])
MAPE=np.sum(abs(test-forecasts)/(test))/(test.shape[0]*test.shape[1])
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

RMSE: 0.22568326799567887 | QLIKE: 0.026825308033247224 | MAE: 0.06849888387071024 | MAPE: 0.18350406678122802
